In [31]:
import pandas as pd

train = pd.read_csv('/content/sample_data/assignment_data_train.csv')
test = pd.read_csv('/content/sample_data/assignment_data_test.csv')

train.head()

import statsmodels.api as sm

x = train.drop(['trips', 'Timestamp'], axis = 1)
y = train[['trips']]
model = sm.OLS(endog = y, exog = x)
modelFit = model.fit()

x_test = test.drop(['Timestamp'], axis = 1)

pred = modelFit.predict(x_test)

print(modelFit.summary())

                            OLS Regression Results                            
Dep. Variable:                  trips   R-squared:                       0.486
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     2765.
Date:                Mon, 17 Feb 2025   Prob (F-statistic):               0.00
Time:                        21:52:52   Log-Likelihood:                -85116.
No. Observations:                8760   AIC:                         1.702e+05
Df Residuals:                    8756   BIC:                         1.703e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
year           3.2348      0.069     47.062      0.0